In [2]:
# -*- encoding=UTF-8
import re, json, random
import requests 
from goose3 import Goose
from goose3.text import StopWordsChinese #导入停用词
from bs4 import BeautifulSoup
from lxml import etree

### 参考
[注释风格](https://zh-google-styleguide.readthedocs.io/en/latest/google-python-styleguide/python_style_rules/)
[翻译api](https://juejin.im/post/5beaac9cf265da614a3a09a9)
[goose提取正文](https://pypi.org/project/goose3/)

可拓展范围：

- [x] gui设计 hs

- [x] 进度显示  hs

- [x] 完善注释  wy

- [x] 完善异常捕获  wy

- [x] 目标字数 (调整gene_shuffle_article函数中para_num=5参数)  wy

- 选择中间语言（判断unicode编码，同时需要调整停用词）  

- [x] 混淆程度  （增加互译次数和翻译软件数,任意一种实现方式均可） rsy

- [x] 添加搜索和翻译引擎（谷歌搜索等） rsy

- 若干页搜索的解决（如果能实现，最好使用yield返回）受反爬虫技术影响，失败 rsy， wy 


In [3]:
def get_access_result(target_url="https://cn.bing.com/", search_word=None):
    """
        Args:
            str:输入搜索字符串

        Returns:
            res:requests返回的response对象

        Raises:
            statuserror:爬取失败

    """
    args={'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'}
    url=target_url
    search={}
    if search_word is not None:
        search['q']=search_word
    try:
        res=requests.get(url, headers=args, params=search, timeout=10) #设置访问时间，超时便访问失败
        res.raise_for_status()
        res.encoding=res.apparent_encoding
        # print(res.text[:1000])
        return res
    # except TimeoutError as tout:
    #     print('Can\'t access to website:'+target_url)
    #     return None
    except Exception as e:
        print(e)
        print("spider failed")
        return None

In [4]:

class Ciba:
    '''使用金山词霸翻译
        
    Attributes:
        url: 金山词霸post方法的url
        headers: 报头参数

    '''

    def __init__(self):
        # self.word = word
        self.url = 'http://fy.iciba.com/ajax.php?a=fy'
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) '
                          'AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.132 Safari/537.36'
        }
        
    # 发送请求
    def request_post(self, word, from_lang='auto', to_lang='auto'):
        """
            Args:
                word:翻译内容
                from_lang：源语言
                to_lang：目标语言

            return：
                None：访问失败
                res.content.decode()：解码后的文本

            Raises：
                statuserror:爬取失败            
        """
        payload = {
            'f': from_lang,
            't': to_lang,
            'w': word
        }
        try:
            res = requests.post(url=self.url, headers=self.headers, data=payload)
            res.raise_for_status()
            # print(res.content.decode())
            return res.content.decode()
        except Exception as e:
            print(e)
            return None
    # 解析数据
    @staticmethod
    def parse_data(data):
        """
            Args：
                data：爬虫解码后的文本

            return：
                dict_data['content']['out']:文字段翻译结果
                dict_data['content']['word_mean']：词语翻译结果
        """
        dict_data = json.loads(data)  #使用json解析
        if 'out' in dict_data['content']:   #文字段翻译
            return dict_data['content']['out']
        elif 'word_mean' in dict_data['content']:  #词语翻译
            return dict_data['content']['word_mean']

    def translate(self, word, from_lang='auto', to_lang='auto'):
        """
            Args:
                word:翻译内容
                from_lang：源语言
                to_lang：目标语言

            return：
                None：访问失败
                res.content.decode()：解码后的文本           
        """
        data = self.request_post(word,from_lang,to_lang)
        if(data==None):
            return ''
        return self.parse_data(data)


In [5]:
class Bing:
    '''使用金山词霸翻译
        
    Attributes:
        url: bing翻译post方法的url
        headers: 报头参数

    '''

    def __init__(self):
        # self.word = word
        self.url = 'https://cn.bing.com/ttranslatev3?'
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) '
                          'AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.132 Safari/537.36'
        }


    # 发送请求
    def request_post(self, word, from_lang='auto-detect', to_lang='auto-detect'):
        """
            Args:
                word:翻译内容
                from_lang：源语言
                to_lang：目标语言

            return：
                None：访问失败
                res.content.decode()：解码后的文本

            Raises：
                statuserror:爬取失败            
        """
        if(from_lang=='zh'):
            from_lang='zh-Hans'
        if(to_lang=='zh'):
            to_lang='zh-Hans'
        data = {
            'fromLang': from_lang,
            'text': word,
            'to': to_lang
        }
        try:
            res = requests.post(url=self.url, headers=self.headers, data=data)
            res.raise_for_status()
            # print(res.content.decode())
            return res.content.decode()
        except Exception as e:
            print(e)
            return None
    # 解析数据
    @staticmethod
    def parse_data(data):
        """
            Args：
                data：爬虫解码后的文本

            return：
                dict_data[0]['translations'][0]['text']:翻译结果
        """
        dict_data = json.loads(data)  #使用json解析
        return dict_data[0]['translations'][0]['text']

    def translate(self, word, from_lang='auto-detect', to_lang='auto-detect'):
        """
            Args:
                word:翻译内容
                from_lang：源语言
                to_lang：目标语言

            return：
                None：访问失败
                res.content.decode()：解码后的文本           
        """
        data = self.request_post(word,from_lang,to_lang)
        if(data==None):
            return ''
        return self.parse_data(data)

In [6]:
class bing_engine:
    '''使用bing搜索
        
    Attributes:
        url: bing搜索根目录url
        headers: 报头参数

    ''' 

    def __init__(self):
        self.url="https://bing.com/"
        self.headers={'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'}

    def search(self, search_word=None):
        """
            Args:
                search_word:输入搜索字符串

            Returns:
                clean_res:返回一组搜索结果和链接

            Raises:
                statuserror:爬取失败

        """
        search_para={}
        if str is not None:
            search_para['q']=search_word
        try:
            res=requests.get(url=self.url, headers=self.headers, params=search_para)
            res.raise_for_status()
            res.encoding=res.apparent_encoding
            # print(res.text[:1000])
            soup=BeautifulSoup(res.text, 'lxml')
            clean_res=[]
            for i in soup.findAll(name=['a'], target='_blank', text=re.compile('[^帮助]')):
                # print(i.string,'\n',i.attrs['href'])
                clean_res.append((i.string, i.attrs['href']))
            return clean_res
        except Exception as e:
            print(e)
            print("spider failed")
            return []

In [7]:
class san60_engine:
    '''使用360搜索
    
    Attributes:
        url: 360搜索根目录url
        headers: 报头参数
    
    '''
    
    def __init__(self):
        self.url="https://www.so.com/s"
        self.headers={'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.61 Safari/537.36'}
        
    def search(self, search_word=None):
        """
            Args:
                search_word:输入搜索字符串

            Returns:
                clean_res:返回一组搜索结果和链接

            Raises:
                statuserror:爬取失败
        """
        search_para={}
        if search_word is not None:
            search_para['q']=search_word
        try:
            res=requests.get(url=self.url, headers=self.headers, params=search_para)
            res.raise_for_status()
            res.encoding=res.apparent_encoding
            
            tree = etree.HTML(res.text)
            url_list=tree.xpath('//h3[@class]/a[@href]/@href')
            for i in range(0,len(url_list)):
                if 'link' in url_list[i]:
                    try:
                        tmp_url = requests.get(url_list[i])
                        # print(tmp_url.text)
                        patt = re.compile('replace(.*?)</script>')
                        url_list[i]=eval(re.findall(patt,tmp_url.text)[0])
                        # print(eval(re.findall(patt,tmp_url.text)[0]))
                    except Exception as e:
                        print(e)
                        print("spider failed")
            content_list=tree.xpath('//h3[@class]/a[@href]/text()')
            clean_res=list(zip(content_list[:-1],url_list[:-1]))
            return clean_res
            
        except Exception as e:
            print(e)
            print("spider failed")
            return None

In [8]:
class sogou_engine:
    '''使用搜狗搜索
    
     Attributes:
        url: bing搜索根目录url
        headers: 报头参数
    '''
    
    def __init__(self):
        self.url0="https://www.sogou.com"
        self.url="https://www.sogou.com/web"
        self.headers={'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.61 Safari/537.36'}
        
    
    def search(self, search_word=None):
        """
            Args:
                search_word:输入搜索字符串

            Returns:
                clean_res:返回一组搜索结果和链接

            Raises:
                statuserror:爬取失败

        """
        search_para={}
        if search_word is not None:
            search_para['query']=search_word
        try:
            res=requests.get(url=self.url, headers=self.headers, params=search_para)
            res.raise_for_status()
            res.encoding=res.apparent_encoding
            tree=etree.HTML(res.text)
            # soup=BeautifulSoup(res.text,'lxml')
            # print(res.url)
            # print(soup.prettify())
            url_list=tree.xpath('//h3/a[@id and @target="_blank" and @href]/@href')
            for i in range(0,len(url_list)):
                if 'link' in url_list[i]:
                    url_list[i]=self.url0+url_list[i]
                    try:
                        tmp_url = requests.get(url_list[i])
                        # print(tmp_url.text)
                        patt = re.compile('replace(.*?)</script>')
                        url_list[i]=eval(re.findall(patt,tmp_url.text)[0])
                        # print(eval(re.findall(patt,tmp_url.text)[0]))
                    except:
                        pass
            content_list=tree.xpath('//h3/a[@id and @target="_blank" and @href]/text()')
            clean_res=list(zip(content_list,url_list))
            return clean_res
        except Exception as e:
            print(e)
            print("spider failed")
            return None

In [9]:
def get_paragrams(search_res):
    """
        Args:
            search_res:返回一组搜索结果和链接

        Returns:
            clean_res:返回所有链接的正文段落

        Raises:
            e:文章段落分割异常
    """
    paras=[]
    goose = Goose({'browser_user_agent': 'Mozilla', 'parser_class':'soup','stopwords_class': StopWordsChinese})  # 设置goose参数
    for  ind, res_elem in enumerate(search_res):
        try:
            res_herf=res_elem[1]
            if get_access_result(target_url=res_herf)==None:  #测试是否可以访问
                print('Can\'t access to website:'+res_herf)
                continue
            article=goose.extract(url=res_herf)  #正文提取 异常处理
            paras.extend(list(article.cleaned_text.split()))  #分割成段
        except Exception as e:
            print("Fail to split paragrams in",res_elem[1], end='  ')
            print(e)
            continue
    return paras

In [10]:
def gene_shuffle_article(search_res, word_limit=800):
    """
        Args:
            search_res:返回一组搜索结果和链接
            word_limit:文章最低字数限制
        Returns:
            article_gene:返回随机打乱后组合的文章

        Raises:
            e:段落组合失败

    """
    paragrams=get_paragrams(search_res)  # 获取段落
    # print(len(paragrams))
    total_paras=len(paragrams) # 总段落
    article_gene, total_words='', 0  ##生成文章与已加入段落数
    st, total_st=set(), 0  ##使用过的段落集合
    try:
        while total_words<=word_limit and total_st<=0.2*total_paras:  #使用不超过20%的段落，字数超过后停止
            i= random.randint(0,total_paras-1)  #生成不重复的随机数 
            if(i not in st):  #不重复
                st.add(i)   #插入集合
                total_words+=len(paragrams[i])  ##总字数增加
                total_st+=1 #总段落数增加
                article_gene+=paragrams[i]+'\n'  # 段落拼接
            else:
                continue #重复则略过
    except Exception as e:
        print(e)
    return article_gene

In [17]:
def rubbish_essay(search_word, search_engine='bing', trans_engine='ciba', word_limit=800, confusion_level=1):
    """
        Args:
            search_word:搜索主题
            search_engine: 搜索引擎
            trans_engine: 翻译引擎
            word_limit: 最少字数
            confusion_level: 混淆程度
        Returns:
            warning: engine doesn't exist or word_limit is too large 
            article_gene:返回随机打乱后组合的文章

    """
    if search_engine == 'bing':  #选择搜索引擎
        s_engine = bing_engine()
    elif search_engine=='360':
        s_engine=san60_engine()
    elif search_engine=='sogou':
        s_engine=sogou_engine()
    else :
        return 'Please use a supported translation engine.'
    search_res = s_engine.search(search_word) 
    article_gene = gene_shuffle_article(search_res, word_limit)
    if trans_engine == 'ciba':  #选择翻译引擎
        t_engine = Ciba()
    elif trans_engine == 'bing':
        t_engine = Bing()
    else:
        return 'Please use a supported translation engine.'
    if len(article_gene)==0 :  #无搜索结果返回
        return 'No result for {}'.format(search_word)
    article_final = ''
    for paragram in article_gene.split():  ## 按段落翻译，防止超字数限制
        paragram_zh=paragram
        while confusion_level:
            confusion_level-=1
            paragram_en = t_engine.translate(  
                word=paragram_zh, from_lang='zh', to_lang='en')  
            paragram_zh= t_engine.translate(
                word=paragram_en, from_lang='en', to_lang='zh')
        article_final +=paragram_zh+'\n'
    if len(article_final)<word_limit: #小于最小字长
        article_final="The required number of words is too large."+article_final
    return article_final

In [29]:
print(rubbish_essay(search_word="抗疫感想",word_limit=800))

在特殊的“战场”上，党员干部要保持“责任田”的良好意图。 爆发是命令，防控是责任.. 目前冠状病毒诊断数量还在上升，比较危险的地方有党员干部的身影.. 一名党员是一面旗帜，要求党员干部首先要为人民群众的生命安全和健康负责，把防空工作作为当前最重要的工作来抓；要有责任心，在自己的岗位上谋政，尽职尽责，把防空的责任放在心里，抵制责任，确保人民群众的安全；把防控工作作为“不忘初心，牢记使命”主题教育的试金石，在战场上践行党的宗旨..
目前，疫情还在发展，值此关键时刻，让勇敢的人们有后盾，让奉献的人们有后劲，让所有努力都奔向胜利的目标，我们还要继续相互支持、紧密配合、坚持到底。我们爱这个国家，爱这座城市，不仅在昨天和今天，更在未来的每一天。真爱和真情就是一种力量。“于你于我，同心同根”。这种力量来自你，来自我们中每一个履行公民义务的人！
有目共睹的“中国速度”，筑牢打赢新冠肺炎疫情阻击战的信心。从1月24日到2月2日，中国在争分夺秒，与时间来了一场“拔河”比赛，短短十天，武汉火神山医院完工并交付使用;2月5日，雷神山医院也交付使用;2月3日到2月4日，武汉建立3所“方舱医院”;从发出应对新冠肺炎的倡议书起，1小时内收到400余份“请战书”;山东大学齐鲁医院紧急组建医疗队驰援湖北的通知发出后，1小时内迅速组建完毕……成立以钟南山院士为组长的新型冠状病毒感染的肺炎疫情联防联控工作机制科研攻关专家组，仅用1周时间完成病毒鉴定和测序。仅用2天时间建成信息报告系统。仅用3天时间初步研发出新冠病毒核酸检测试剂盒。由此可见，与时间赛跑，如此有目共睹的“中国速度”，何尝不能筑牢打赢战“疫”的信心。
经过这次志愿者服务的经历，我深深的体会到党和国家以及人民群众之间相互配合的重要性。此外，我觉得自告奋勇，义无反顾应该是当代大学生应该发扬的品质，而不是抱着事不关己，高高挂起的心态。当国家遇到困难的时候，我们需要积极的参与进去，不逃避，不退缩，勇往直前。家喻户晓的是在这场战役中，我们每个人都奉献出了自己的微薄之力。俗话说，守得云开见月明，我相信只要我们众志成城，胜利终将会到来。加油，中国!加油，武汉!



In [31]:
print(rubbish_essay(search_word="xakdslfja",word_limit=200))

empty range for randrange() (0,0, 0)
No result for xakdslfja


In [30]:
print(rubbish_essay(search_word="青年大学习感想",word_limit=1200))

HTTPConnectionPool(host='wenku.baidu.comhttps', port=80): Max retries exceeded with url: //wenku.baidu.com/ndcore/browse/sub?isType=10&fr=view (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000289F1140F28>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))
spider failed
Can't access to website:http://wenku.baidu.comhttps://wenku.baidu.com/ndcore/browse/sub?isType=10&fr=view
HTTPConnectionPool(host='wenku.baidu.comhttps', port=80): Max retries exceeded with url: //wenku.baidu.com/ndcore/browse/sub?isType=10&fr=view (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000289F11867B8>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))
spider failed
Can't access to website:http://wenku.baidu.comhttps://wenku.baidu.com/ndcore/browse/sub?isType=10&fr=view

通过对“青年大学习”网上主题团课第十期的学习，我了解到了国家对青少年的重视，从小时候的“我们是祖国的花朵”的兴奋，到现在“少年强则国强，少年富则国富”的责任，我们也在慢慢同国家响应着少年的成长。习主席说：“青年工作，抓住的是当下，传承的是根脉。”的确青少年承载的

In [11]:
print(rubbish_essay(search_word="青年大学习感想",search_engine="sogou",word_limit=1200))

[('16篇- 爱师网', 'https://www.is97.com/doc-174959-1.html'), ('范文10篇- 豆丁网', 'https://www.docin.com/p-2159284463.html'), ('感悟范文6篇- 道客巴巴', 'http://www.doc88.com/p-8149127255981.html'), ('| ', 'http://www.360kuai.com/pc/943d5a00e76bc2e97?cota=4&tj_url=so_rec&sign=360_57c3bbd1&refer_scene=so_1'), ('2018', 'https://www.docin.com/p-2129662326.html'), ('- 豆丁网', 'https://www.doc88.com/p-3009162559945.html'), ('2019范文- 道客巴巴', '//kch.so.com/result?kid=22&uiid=a8c3a57c3cd56bc6eb2fda5ab09ba8f4&q=%E9%9D%92%E5%B9%B4%E5%A4%A7%E5%AD%A6%E4%B9%A0%E6%84%9F%E6%83%B3&title=%E9%9D%92%E5%B9%B4%E5%85%9A%E5%91%98%E5%AD%A6%E4%B9%A0%E5%8D%81%E4%B9%9D%E5%A4%A7%E6%8A%A5%E5%91%8A%E5%BF%83%E5%BE%97%E4%BD%93%E4%BC%9A&category=education&srcg=kch_menu&src=kch_title'), ('党员', 'https://fanwen.chazidian.com/fanwen488730/'), ('十九', 'http://www.xuexila.com/yc/c179533.html'), ('报告', 'https://www.xuexila.com/yc/c396916.html')]
Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\Lenovo\AppData\Lo

In [12]:
print(rubbish_essay(search_word="青年大学习感想",search_engine="sogou",word_limit=24534546))

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\Lenovo\AppData\Local\Temp\jieba.cache
Loading model cost 3.247 seconds.
Prefix dict has been built successfully.
The required number of words is too large.
学习习近平总书记讲话，二是要多思考。在阅读大量材料的基础上，要不断深入思考，辨析各项政策、讲话推出的时代背景和时事背景，要学会对相关材料进行横向、纵向的比较和分析，从而掌握更明确的政策脉络，开阔自己的眼界视野和格局。
作为一名团员，能够说这次的学习让我又一次正视了自我的团员身份，同时也如当头棒喝，提示着我就应重新审视团员形象和职责。
习近平总书记特别强调：共青团必须站在理想信念的制高点上，引领青年树立理想。对于每一个人来说，信念都是很重要的，树立一个正确的理想信念，会使我们不在迷茫，会使我们充满动力。共青团是为青年工作的，所以共青团必须担起引领青年树立正确理想信念的重担。
青年，是国家的未来和民族的希望，是社会上最富有朝气、最富有创造性、最富有生命力的群体。
青年，是国家的未来和民族的希望，是社会上最富有朝气、最富有创造性、最富有生命力的群体，通过这次青年大学习我深刻的认识到作为一名共青团员，首先要端正思想，提高思想觉悟水平。在学习和生活上树立起作为共青团员就应起到带头和模范作用，其次用心参与学校组织的各项团员活动，在活动中学习理论，在活动中实践理论，这样才能做到学以致用，也贴合马克思的理论与实践相结合的原则。我们是新世纪的青年一代，我们是八九点钟的太阳，我们有国家领导人和全国人民对我们的殷切期望。我们要努力提高自身思想素质，弘扬社会主义道德风尚，认真遵守学院各项规章制度，自觉履行团员的义务，发挥团员的模范作用。
大学生活是短暂和美丽的。我们不应虚度如此黄金岁月，不能让大学过得碌碌无为而遗憾一生。我们要着眼于自身实际，不断追求进步，明确自我读大学的目标，明确对自我的定位，明确要培养哪方面的潜力，明确要怎样提高自我的综合素质，明确学习的计划步骤，等等，逐项明确以后，并付诸行动中，才能使自我经

In [18]:
print(rubbish_essay(search_word="华中师范大学最好吃的食堂",word_limit=600))

HTTPConnectionPool(host='list', port=80): Max retries exceeded with url: /108 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000023D3449DD30>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))
spider failed
Can't access to website:http://jingyan.baidu.com//list/108
HTTPConnectionPool(host='list', port=80): Max retries exceeded with url: /115 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000023D3449DDA0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))
spider failed
Can't access to website:http://jingyan.baidu.com//list/115
 名片解读：二号楼和一号楼，建于1956年，华中师范大学由谭华林迁至贵子山首批老楼之一，2013年被列为武汉市一级保护历史建筑.. 现为我校艺术学院办公教学楼..
名片解读：华大图书馆的前身是创建于1903年的文华公书林，迄今已有百年的历史。1951年，由原华中大学、中华大学、中原大学教育学院三校的图书馆合并成为华中师范学院图书馆，1985年更名为华中师范大学图书馆。现在图书馆由总馆（新馆）和老馆组成。老馆建于1961年，面积为9000平方米，是华中师范大学从昙华林迁到桂子山上的首批老建筑之一。新馆2011年5月16日正式开放，总建筑面积约30357平方米，建有地上9层、地下2层，建筑高度约40米，可同时容纳5000人学习阅览。新馆和老馆中间由一条地下文化长廊连通，馆舍总面积达39，357平方米。
：一